In [ ]:
# import necessary libraries, filter warnings
import flowermd
import gsd
import gsd.hoomd
import hoomd
import matplotlib.pyplot as plt
import mbuild as mb
import numpy as np
import warnings
from cmeutils.visualize import FresnelGSD
from flowermd.base import Pack, Simulation, System, Molecule
from flowermd.base.forcefield import BaseHOOMDForcefield
from flowermd.library import KremerGrestBeadSpring, LJChain
from flowermd.library.forcefields import BeadSpring
from flowermd.utils import get_target_box_number_density
from mbuild.compound import Compound
from mbuild.lattice import Lattice
import unyt as u
warnings.filterwarnings('ignore')

## Define Flake Geometry

In [ ]:
class Graphene(System):
    def __init__(
        self,
        x_repeat,
        y_repeat,
        n_layers,
        base_units=dict(),
        periodicity=(True, True, False),
    ):
        surface = mb.Compound(periodicity=periodicity)
        a = 3**.5



        
        lattice = Lattice(
            lattice_spacing=[a,a,a],
            lattice_vectors=  [[a,0,0],[a/2,3/2,0],[0,0,1]],
            lattice_points={"A": [[1/3,1/3,0], [2/3, 2/3, 0]]},
        ) 
        Flakium = Compound(name="F", element="F") # defines a carbon atom that will be used to populate lattice points
        layers = lattice.populate(
            compound_dict={"A": Flakium}, x=x_repeat, y=y_repeat, z=n_layers
        ) # populates the lattice using the previously defined carbon atom for every "A" site, repeated in all x,y, and z directions
        surface.add(layers) # adds populated carbon lattice layers to the 'surface' compound, which represents our graphene structure 
        surface.freud_generate_bonds("F", "F", dmin=0.9, dmax=1.1) # generates bonds depending on input distance range, scales with lattice
        surface_mol = Molecule(num_mols=1, compound=surface) # wraps into a Molecule object, creating "1" instance of this molecule

        super(Graphene, self).__init__(
            molecules=[surface_mol],
            base_units=base_units,
        )

    def _build_system(self):
        return self.all_molecules[0]

## Define Forcefield for WCA Interactions
Essentially zero attraction; pure repulsion.

In [ ]:
ff = BeadSpring(
    r_cut=2**(1/6),  
    beads={
        "A": dict(epsilon=1.0, sigma=1.0),  # chains
        "F": dict(epsilon=1.0, sigma=1.0),  # flakes
    },
    bonds={
        "F-F": dict(r0=1.0, k=1000),
        "A-A": dict(r0=1.0, k=1000.0),  # increased k to avoid chain collapse
    },
    angles={
        "A-A-A": dict(t0=2* np.pi / 3., k=100.0),   # moderate stiffness for chains
        "F-F-F": dict(t0=2 * np.pi / 3., k=5000),
    },
    dihedrals={
        "A-A-A-A": dict(phi0=0.0, k=0, d=-1, n=2), #need to turn this on later, messed up with straight chains
        "F-F-F-F": dict(phi0=0.0, k=500, d=-1, n=2),
    }
)

# Define Simulation

In [ ]:
def run_simulation(N_chains=40, initial_dens=0.001, final_dens=0.3, N_flakes=10, 
                   chain_length=10, dt=0.0005, temp=3, device='CPU'):
    """
    Runs an entropy-driven aggregation simulation on HOOMD with the specified parameters.

    Parameters:
    - N_chains: Number of polymer chains.
    - initial_dens: Initial density of the system.
    - final_dens: Final target density for the system.
    - N_flakes: Number of flakes.
    - chain_length: Length of each chain.
    - dt: Time step size.
    - temp: Temperature.
    - device: 'CPU' or 'GPU' for the simulation device.
    """

    # Set device (CPU or GPU), defaulted to CPU
    if device == 'GPU':
        device = hoomd.device.GPU()
    else:
        device = hoomd.device.CPU()

    # Initialize the chain and sheet
    kg_chain = LJChain(lengths=chain_length, num_mols=N_chains)
    sheet = Graphene(x_repeat=5, y_repeat=5, n_layers=1, periodicity=(False, False, False))
    
    # Initialize the system
    system = Pack(molecules=[Molecule(compound=sheet.all_molecules[0], num_mols=N_flakes), kg_chain], 
                  density=initial_dens, packing_expand_factor=6, seed=2)
    
    # Get the target box size based on the final density
    target_box = get_target_box_number_density(density=final_dens * u.Unit("nm**-3"), n_beads=(500 + (N_chains * 10)))

    # Prepare file output
    gsd = f"{N_chains}_{chain_length}mer{N_flakes}f_{dt}dt.gsd"
    log = f"{N_chains}_{chain_length}mer{N_flakes}f_{dt}dt.txt"

    # Initialize the simulation object
    sim = Simulation(initial_state=system.hoomd_snapshot, forcefield=ff.hoomd_forces, 
                     device=device, dt=dt, gsd_write_freq=1000, log_file_name=log, gsd_file_name=gsd)

    # Run the simulation with volume update and thermalization
    sim.run_update_volume(final_box_lengths=target_box, kT=6.0, n_steps=5e6, 
                          tau_kt=100 * sim.dt, period=10, thermalize_particles=True)

    # Run NVT ensemble simulation
    sim.run_NVT(n_steps=5e6, kT=temp, tau_kt=dt * 100)
    sim.flush_writers() # Flush data into output files

    # Visualize results - TODO, add as function parameters or prompt options after simulation
    sim_visualizer = FresnelGSD(gsd_file=gsd, frame=-1, view_axis=(1, 1, 1))
    sim_visualizer.view()

In [ ]:
# example function usage
# run_simulation(30,0.001,0.3,10,10,0.0005,3,'CPU');

In [ ]:
# Add command line usage
if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser(description="Run HOOMD simulation with custom parameters.")
    parser.add_argument("--N_chains", type=int, default=40)
    parser.add_argument("--initial_dens", type=float, default=0.001)
    parser.add_argument("--final_dens", type=float, default=0.3)
    parser.add_argument("--N_flakes", type=int, default=10)
    parser.add_argument("--chain_length", type=int, default=10)
    parser.add_argument("--dt", type=float, default=0.0005)
    parser.add_argument("--temp", type=float, default=3)
    parser.add_argument("--device", choices=["CPU", "GPU"], default="CPU")

    args = parser.parse_args(args=[])

    run_simulation(
        N_chains=args.N_chains,
        initial_dens=args.initial_dens,
        final_dens=args.final_dens,
        N_flakes=args.N_flakes,
        chain_length=args.chain_length,
        dt=args.dt,
        temp=args.temp,
        device=args.device
    )